In [1]:
!pip install flask flask-ngrok pandas numpy haversine ortools


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

orders_df = pd.read_csv('/content/drive/MyDrive/orders.csv')
warehouses_df = pd.read_csv('/content/drive/MyDrive/warehouses.csv')

orders_df.head()


,order_id,lat,lon,item,qty,time_placed
0,o0,12.915881,77.698118,B,1,2025-07-13T18:22:42.050177
1,o1,13.049441,77.551380,A,2,2025-07-13T18:22:42.050177
2,o2,12.918553,77.633675,B,1,2025-07-13T18:22:42.050177
3,o3,12.882885,77.618167,B,1,2025-07-13T18:22:42.050177
4,o4,12.963623,77.645884,C,1,2025-07-13T18:22:42.050177


In [5]:
!pip install pyngrok flask pandas numpy haversine ortools


In [7]:
pip install flask pandas numpy haversine scikit-learn ortools

In [8]:
import pandas as pd
import random

warehouses = [f"w{i}" for i in range(1, 21)]
items = ['soap', 'shampoo', 'toothpaste', 'biscuits', 'mosquito net', 'chocolate box', 'diyas', 'light bulb', 'washing powder', 'sanitizer']

# Generate inventory for 20 warehouses with 5–10 random items per warehouse
inventory_rows = []
for wh in warehouses:
    available_items = random.sample(items, k=random.randint(5, 10))
    for item in available_items:
        stock = random.randint(5, 50)
        inventory_rows.append({'wh_id': wh, 'item': item, 'stock': stock})

# Save to CSV
df_inventory = pd.DataFrame(inventory_rows)
df_inventory.to_csv('/content/inventory.csv', index=False)

# Optional: Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
df_inventory.to_csv('/content/drive/MyDrive/inventory.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!ngrok config add-authtoken 2zpeRxSKY8XVEvSurdZrJzQkFFd_oPjTmbE4Umvm663H1TfF


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from haversine import haversine
import datetime

# Initialize Flask app
app = Flask(__name__)
AVERAGE_SPEED_KMPH = 30

# Load inventory data
inventory_df = pd.read_csv('inventory.csv')  # Make sure this CSV is in the same folder

def get_closest_wh_with_stock(order_items, order_latlon, wh_df):
    sorted_wh = wh_df.copy()
    sorted_wh['distance'] = sorted_wh.apply(
        lambda wh: haversine(order_latlon, (wh['lat'], wh['lon'])), axis=1
    )
    sorted_wh = sorted_wh.sort_values('distance')

    for _, wh in sorted_wh.iterrows():
        wh_id = wh['wh_id']
        has_all_items = True
        for item in order_items:
            stock_row = inventory_df[(inventory_df['wh_id'] == wh_id) & (inventory_df['item'] == item.strip())]
            if stock_row.empty or stock_row.iloc[0]['stock'] < 1:
                has_all_items = False
                break
        if has_all_items:
            return wh
    return None

@app.route('/route', methods=['POST'])
def route():
    data = request.get_json()
    orders = pd.DataFrame(data['orders'])        # order_id, lat, lon, items
    warehouses = pd.DataFrame(data['warehouses'])  # wh_id, lat, lon

    coords = orders[['lat', 'lon']].values
    k = min(len(warehouses), len(orders))  # Use min of orders and warehouses
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    orders['cluster'] = kmeans.fit_predict(coords)

    dispatch_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=2)).isoformat()

    routes = []
    for cluster_id in orders['cluster'].unique():
        cluster_orders = orders[orders['cluster'] == cluster_id]

        for _, order in cluster_orders.iterrows():
            order_latlon = (order['lat'], order['lon'])
            order_items = order['items'].split(',')

            assigned_wh = get_closest_wh_with_stock(order_items, order_latlon, warehouses)

            if assigned_wh is None:
                result = {
                    'order_id': order['order_id'],
                    'assigned_wh': None,
                    'status': 'out_of_stock',
                    'message': 'No warehouse has all required items'
                }
            else:
                dist_km = haversine(order_latlon, (assigned_wh['lat'], assigned_wh['lon']))
                eta_minutes = round((dist_km / AVERAGE_SPEED_KMPH) * 60)
                result = {
                    'order_id': order['order_id'],
                    'assigned_wh': assigned_wh['wh_id'],
                    'dispatch_time': dispatch_time,
                    'eta': f'{eta_minutes} mins',
                    'status': 'scheduled'
                }

            routes.append(result)

    return jsonify({'routes': routes})

if __name__ == '__main__':
    port = 5000
    public_url = ngrok.connect(port)
    print(f"🚚 Public API running at: {public_url}/route")
    app.run(port=port)


🚚 Public API running at: NgrokTunnel: "https://ca282310a738.ngrok-free.app" -> "http://localhost:5000"/route
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 19:00:34] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Jul/2025 19:02:25] "POST /route HTTP/1.1" 200 -
